In [19]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)

In [2]:
#Need to find out if all duplicates are nonzeros
#Tostipanc
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Tostipanctits")
#Upload matrices
data_tosti_panctits_matrix <- readMM("data_tosti_panctits_matrix.mtx")
#Upload original gene list
data_tosti_panctits_genes <- read.delim("data_tosti_panctits_genes.tsv")
data_tosti_panctits_genes <- data_tosti_panctits_genes$genes
data_tosti_panctits_cellIDs <- read.delim("data_tosti_panctits_cellIDs.tsv")
rownames(data_tosti_panctits_matrix) <- data_tosti_panctits_genes
colnames(data_tosti_panctits_matrix) <- data_tosti_panctits_cellIDs$cellIDs

In [21]:
#Upload duplicates list
data_tosti_panctits_genes_dup <- read.xlsx("data_tosti_panctits_geneIDs_forsum.xlsx")

In [23]:
# Initialize an empty list to store the sums
sums_list <- list()

# Iterate over each gene in your list
for (gene in data_tosti_panctits_genes_dup$Old) {
  # Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(data_tosti_panctits_matrix[rownames(data_tosti_panctits_matrix) == gene, ])
  
  # Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [ ]:
sums_list

In [16]:
dup_sum <- do.call(rbind, lapply(sums_list, as.data.frame))

In [17]:
dup_sum$genes <- rownames(dup_sum)

In [18]:
write.xlsx(list(dup_sum), "data_tosti_panctits_geneIDs_dup_sum.xlsx")

In [25]:
#Upload manually editted list
data_tosti_panctits_genes <- read.xlsx("data_tosti_panctits_geneIDs_int_removed_manualedit.xlsx")

In [27]:
#Plug new genes to the matrix
rownames(data_tosti_panctits_matrix) <- data_tosti_panctits_genes$New

In [28]:
#Check for duplicates. Do they match the genes to be concatenated?
duplicate_genes <- duplicated(rownames(data_tosti_panctits_matrix))
cat("Duplicate gene IDs:\n")
print(rownames(data_tosti_panctits_matrix)[duplicate_genes])
duplicated <- rownames(data_tosti_panctits_matrix)[duplicate_genes]
#They match

Duplicate gene IDs:
 [1] "BTBD8"        "GPR89B"       "NBPF11"       "NBPF15"       "HNRNPU"      
 [6] "RRM2"         "TRAF3IP2-AS1" "CASTOR2"      "CFAP47"       "CFAP47"      
[11] "TSPAN7"       "DLGAP2"       "AGO2"         "FAM27E2"      "ZNG1F"       
[16] "MRC1"         "NSUN6"        "C10orf67"     "CCDC7"        "PTPN20"      
[21] "AGAP4"        "WASHC2A"      "LCOR"         "MICAL2"       "ARHGAP42"    
[26] "GRAMD1B"      "PLEKHG7"      "SUCLA2"       "MIA2"         "SYNE3"       
[31] "RPS17"        "ZFHX3"        "NAA38"        "GRIK5"        "PSG9"        
[36] "AREG"        


In [29]:
#Filter zeros 
row_sums <- rowSums(data_tosti_panctits_matrix)
names_with_zeros <- names(row_sums[row_sums == 0])
length(names_with_zeros)

[1] 36

In [30]:
filtered_data_tosti_panctits_matrix <- data_tosti_panctits_matrix[row_sums != 0, ]

In [31]:
str(data_tosti_panctits_matrix)
str(filtered_data_tosti_panctits_matrix)

Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:6907682] 5 19 22 44 45 53 66 68 69 75 ...
  ..@ j       : int [1:6907682] 0 0 0 0 0 0 0 0 0 0 ...
  ..@ Dim     : int [1:2] 24782 2666
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:24782] "RP11-34P13.7" "AL627309.1" "AP006222.2" "RP4-669L17.10" ...
  .. ..$ : chr [1:2666] "AAACGAAAGCATCCTA_1" "AAACGAACACTCATAG_1" "AAACGAAGTATTCTCT_1" "AAACGAAGTTCTTAGG_1" ...
  ..@ x       : num [1:6907682] 1 1 1 1 5 1 1 1 3 2 ...
  ..@ factors : list()
Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:6907682] 5 19 22 44 45 53 66 68 69 75 ...
  ..@ j       : int [1:6907682] 0 0 0 0 0 0 0 0 0 0 ...
  ..@ Dim     : int [1:2] 24746 2666
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:24746] "RP11-34P13.7" "AL627309.1" "AP006222.2" "RP4-669L17.10" ...
  .. ..$ : chr [1:2666] "AAACGAAAGCATCCTA_1" "AAACGAACACTCATAG_1" "AAACGAAGTATTCTCT_1" "AAACGAAGTTCTTAGG_1" ...
  ..@ x       : num [1:6907682] 1 1 1 1 5 1 1

In [32]:
#Concatenating replicates
filtered_data_tosti_panctits_matrix <- as(filtered_data_tosti_panctits_matrix, "dgCMatrix")

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



In [33]:
gene_names <- rownames(filtered_data_tosti_panctits_matrix)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)

In [34]:
aggregated_counts <- rowsum(as.matrix(filtered_data_tosti_panctits_matrix), group = gene_factors)

In [36]:
str(aggregated_counts)

 num [1:24710, 1:2666] 0 0 0 0 0 1 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:24710] "RP11-34P13.7" "AL627309.1" "AP006222.2" "RP4-669L17.10" ...
  ..$ : chr [1:2666] "AAACGAAAGCATCCTA_1" "AAACGAACACTCATAG_1" "AAACGAAGTATTCTCT_1" "AAACGAAGTTCTTAGG_1" ...


In [37]:
#Creating a seurat object
#Load metadata and dim reductions
data_tosti_panctits_metadata <- read_csv("data_tosti_panctits_metadata_final.csv")

New names:
• `` -> `...1`
• `...1` -> `...2`
Rows: 2666 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (13): ...2, sex, condition, study, cells, lineage, subclass, patientID, ...
dbl  (5): ...1, nCount_RNA, nFeature_RNA, percent.mt, age
lgl  (1): smoking

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [38]:
data_tosti_panctits_metadata <- as.data.frame(data_tosti_panctits_metadata)

In [40]:
rownames(data_tosti_panctits_metadata) <- data_tosti_panctits_metadata$`...2`

In [41]:
data_tosti_panctits_metadata$`...2` <- NULL
data_tosti_panctits_metadata$`...1` <- NULL

In [43]:
data_tosti_panctits_seurat <- CreateSeuratObject(counts = aggregated_counts, meta.data = data_tosti_panctits_metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [45]:
save(data_tosti_panctits_seurat, file = "data_tosti_panctits_seurat.RData")